## Programación del filtro de Kalman

#### Creamos la clase Kalman que estima los parámetros con Máxima Verosimilitud

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
from scipy.optimize import minimize

class KalmanMLE:
    def __init__(self, mid_price, volume_diff, best_bid=None, best_ask=None):
        """
        Inicializa el estimador MLE para el filtro de Kalman.

        Parámetros:
        - mid_price: Serie temporal del mid price.
        - volume_diff: Serie temporal de la diferencia de volúmenes (bid - ask).
        - best_bid: Serie temporal del mejor precio bid (opcional).
        - best_ask: Serie temporal del mejor precio ask (opcional).
        """
        self.mid_price = mid_price.values
        self.volume_diff = volume_diff.values
        self.best_bid = best_bid.values if best_bid is not None else None
        self.best_ask = best_ask.values if best_ask is not None else None
        self.observations = np.column_stack([mid_price, volume_diff])

        # Matriz de transición (4x4)
        self.transition_matrix = np.array([
            [1, 0, 1, 0],  # p_t = p_{t-1} + v_{t-1}
            [0, 1, 0, 1],  # v_t = v_{t-1}
            [0, 0, 1, 0],  # d_t = d_{t-1} + s_{t-1}
            [0, 0, 0, 1]   # s_t = s_{t-1}
        ])

        # Matriz de observación (2x4)
        self.observation_matrix = np.array([
            [1, 0, 0, 0],  # Observamos p_t
            [0, 1, 0, 0]   # Observamos d_t
        ])

    def log_likelihood(self, params):
        """
        Calcula la log-verosimilitud negativa para los parámetros dados.

        Parámetros:
        - params: Lista con [Q, R_price, R_volume].
        """
        Q, R_price, R_volume = params
        observation_covariance = np.diag([R_price, R_volume])

        # Estado inicial
        initial_state_mean = [self.mid_price[0], 0, self.volume_diff[0], 0]
        initial_state_covariance = np.eye(4)

        # Crear el filtro
        kf = KalmanFilter(
            transition_matrices=self.transition_matrix,
            observation_matrices=self.observation_matrix,
            initial_state_mean=initial_state_mean,
            initial_state_covariance=initial_state_covariance,
            transition_covariance=Q * np.eye(4),
            observation_covariance=observation_covariance
        )

        # Calcular la log-verosimilitud
        log_likelihood_value = kf.loglikelihood(self.observations)
        return -log_likelihood_value  # Minimizamos la log-verosimilitud negativa

    def fit(self):
        """
        Estima los parámetros óptimos (Q, R_price, R_volume) usando MLE.
        """
        # Valores iniciales para Q, R_price, R_volume
        initial_params = [0.01, 0.1, 10]

        # Optimización
        result = minimize(
            self.log_likelihood,
            initial_params,
            method='L-BFGS-B',
            bounds=[(1e-5, 1), (1e-5, 1), (1e-5, 50)]
        )

        # Parámetros optimizados
        self.Q_opt, self.R_price_opt, self.R_volume_opt = result.x
        self.observation_covariance_opt = np.diag([self.R_price_opt, self.R_volume_opt])

        return {
            'Q': self.Q_opt,
            'R_price': self.R_price_opt,
            'R_volume': self.R_volume_opt
        }

    def get_optimal_filter(self):
        """
        Devuelve un filtro de Kalman con los parámetros óptimos estimados.
        """
        initial_state_mean = [self.mid_price[0], 0, self.volume_diff[0], 0]
        initial_state_covariance = np.eye(4)

        optimal_kf = KalmanFilter(
            transition_matrices=self.transition_matrix,
            observation_matrices=self.observation_matrix,
            initial_state_mean=initial_state_mean,
            initial_state_covariance=initial_state_covariance,
            transition_covariance=self.Q_opt * np.eye(4),
            observation_covariance=self.observation_covariance_opt
        )

        return optimal_kf

    def plot_results(self):
        """
        Grafica los resultados del filtro de Kalman con los parámetros óptimos.
        """
        # Obtener el filtro óptimo
        optimal_kf = self.get_optimal_filter()

        # Aplicar el filtro
        states, _ = optimal_kf.filter(self.observations)

        # Crear un DataFrame para los resultados
        df_results = pd.DataFrame({
            'mid_price': self.mid_price,
            'volume_diff': self.volume_diff,
            'estimated_price': states[:, 0],
            'estimated_trend': states[:, 1],
            'estimated_volume_diff': states[:, 2],
            'estimated_volume_trend': states[:, 3]
        })

        # Añadir best_bid y best_ask si están disponibles
        if self.best_bid is not None and self.best_ask is not None:
            df_results['best_bid'] = self.best_bid
            df_results['best_ask'] = self.best_ask

        # Graficar
        plt.figure(figsize=(14, 10))

        # Gráfica del precio
        plt.subplot(2, 1, 1)
        plt.plot(df_results['mid_price'], label='Mid Price Observado', alpha=0.5, color='gray')
        if 'best_bid' in df_results and 'best_ask' in df_results:
            plt.plot(df_results['best_bid'], label='Best Bid', alpha=0.7, color='green')
            plt.plot(df_results['best_ask'], label='Best Ask', alpha=0.7, color='red')
        plt.plot(df_results['estimated_price'], label='Precio Estimado (Kalman)', linewidth=2, color='blue')
        plt.legend()
        plt.title('Precio, Best Bid/Ask y Tendencia Estimados')

        # Gráfica de la diferencia de volúmenes
        plt.subplot(2, 1, 2)
        plt.plot(df_results['volume_diff'], label='Diferencia de Volúmenes Observada', alpha=0.5, color='purple')
        plt.plot(df_results['estimated_volume_diff'], label='Diferencia de Volúmenes Estimada (Kalman)', linewidth=2, color='orange')
        plt.legend()
        plt.title('Presión de Compra/Venta y Tendencia')

        plt.tight_layout()
        plt.show()


: 

#### Generamos unos datos simulados

In [ ]:
# Datos simulados del LOB
np.random.seed(42)
n = 100  # Número de observaciones
true_price = np.cumsum(np.random.normal(0, 0.5, n)) + 100  # Precio real
mid_price = true_price + np.random.normal(0, 0.3, n)     # Mid-price observado
spread = 1  # Spread típico
best_bid = mid_price - spread / 2  # Precio bid
best_ask = mid_price + spread / 2  # Precio ask
bid_volume = np.random.poisson(50, n)  # Volumen bid
ask_volume = np.random.poisson(50, n)  # Volumen ask
volume_diff = bid_volume - ask_volume  # Diferencia de volúmenes

# DataFrame
df = pd.DataFrame({
    'mid_price': mid_price,
    'best_bid': best_bid,
    'best_ask': best_ask,
    'bid_volume': bid_volume,
    'ask_volume': ask_volume,
    'volume_diff': volume_diff
})
print(df.head())

#### Aplicamos el filtro de Kalman a los datos

In [ ]:
# Estimación de parámetros con MLE
mle_estimator = KalmanMLE(
    mid_price=df['mid_price'],
    volume_diff=df['volume_diff'],
    best_bid=df['best_bid'],
    best_ask=df['best_ask']
)

# Ajustar parámetros
optimal_params = mle_estimator.fit()
print(f"Parámetros óptimos: Q={optimal_params['Q']:.4f}, R_price={optimal_params['R_price']:.4f}, R_volume={optimal_params['R_volume']:.4f}")

# Graficar resultados
mle_estimator.plot_results()
